In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
from torch import optim
from tqdm import tqdm
import torch.utils.data as data
from sklearn.metrics import roc_auc_score, average_precision_score




In [2]:
train_label = pd.read_csv('./train_label.txt', header=None).to_numpy()
test_label = pd.read_csv('./test_label.txt', header=None).to_numpy()
train_image = np.load('./train_image.npy')
test_image = np.load('./test_image.npy')

In [3]:
class CustomDataset(data.Dataset):
    def __init__(self, img, label):
        super(CustomDataset, self).__init__()
        self.images = img
        self.label = label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img = self.images[index]
        img = np.stack([img, img, img])
        labels = self.label[index]

        return img, labels
    

In [4]:
train_dataset = CustomDataset(train_image, train_label[:,1:])
test_dataset = CustomDataset(test_image, test_label[:,1:])

In [5]:
device = torch.device('cuda:3')
# model = models.resnet50(pretrained=True)
# model.fc = nn.Linear(model.fc.in_features, 14)
# model.to(device)

# lr = 0.1
# num_epochs=5
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# loss_function = nn.BCEWithLogitsLoss().to(device)

train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for epoch in range(num_epochs):
    print(epoch)
    for i, data in enumerate(tqdm((train_dataloader))):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs.float())
        train_loss = loss_function(outputs, labels.float())
        train_loss.backward()
        optimizer.step()

    for i, data in enumerate(tqdm(test_dataloader)):
        imgs, _ = data
        imgs = imgs.to(device)
        predict = torch.sigmoid(model(imgs.float()))
        if i==0:
            prediction = predict
        else:
            prediction = torch.cat([prediction, predict], dim=0)

    auroc_test_macro = roc_auc_score(test_label[:,1:], prediction, multi_class='ovr', average='macro')
    auprc_test_macro = average_precision_score(test_label[:,1:], prediction, average='macro')
    auroc_test_micro = roc_auc_score(test_label[:,1:], prediction, multi_class='ovr', average='micro')
    auprc_test_micro = average_precision_score(test_label[:,1:], prediction, average='micro')
    print(f'AUROC_MACRO:{auroc_test_macro}\nAUPRC_MACRO:{auprc_test_macro}\nAUROC_MICRO:{auroc_test_micro}\nAUPRC_MICRO:{auprc_test_micro}\n')


In [6]:
model = torch.load('./model')
model.to(device)
for i, data in enumerate(tqdm(test_dataloader)):
        imgs, _ = data
        imgs = imgs.to(device)
        predict = torch.sigmoid(model(imgs.float())).detach().cpu()
        if i==0:
            prediction = predict
        else:
            prediction = torch.cat([prediction, predict], dim=0)

auroc_test_macro = roc_auc_score(test_label[:,1:], prediction, multi_class='ovr', average='macro')
auprc_test_macro = average_precision_score(test_label[:,1:], prediction, average='macro')
auroc_test_micro = roc_auc_score(test_label[:,1:], prediction, multi_class='ovr', average='micro')
auprc_test_micro = average_precision_score(test_label[:,1:], prediction, average='micro')
print(f'AUROC_MACRO:{auroc_test_macro}\nAUPRC_MACRO:{auprc_test_macro}\nAUROC_MICRO:{auroc_test_micro}\nAUPRC_MICRO:{auprc_test_micro}\n')

100%|██████████| 210/210 [00:08<00:00, 23.42it/s]


In [7]:
auroc_test_macro = roc_auc_score(test_label[:,1:], prediction, multi_class='ovr', average='macro')
auprc_test_macro = average_precision_score(test_label[:,1:], prediction, average='macro')
auroc_test_micro = roc_auc_score(test_label[:,1:], prediction, multi_class='ovr', average='micro')
auprc_test_micro = average_precision_score(test_label[:,1:], prediction, average='micro')
print(f'AUROC_MACRO:{auroc_test_macro}\nAUPRC_MACRO:{auprc_test_macro}\nAUROC_MICRO:{auroc_test_micro}\nAUPRC_MICRO:{auprc_test_micro}\n')

AUROC_MACRO:0.7350613373584487
AUPRC_MACRO:0.2898902362939529
AUROC_MICRO:0.8541308518655156
AUPRC_MICRO:0.5099094573015348

